In [1]:
initial_path = 'peptide-QML/'
# initial_path = '../'

In [2]:
day = "0830"
name_notebook = day + "-sh_em_1qm_loop_v2.ipynb"

#ask the user if that is correct, if he says yes kkep going, otherwise stop
import sys
answer = input("{} is the name of the notebook | Is that correct? (y/n): ".format(name_notebook))
if answer != "y":
    print("Please, change the name of the notebook")
    sys.exit()

In [3]:
import numpy as np

In [4]:
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import functions as f
from my_code import pytorch_model as ptm
from my_code import quantum_nodes as qn
from my_code import pytorch_wrappers as pw

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Data

In [5]:
data_file_path = initial_path + 'data/Scores/PET/generated/bb14_Strings_Energies_10_000_4_aa.txt'  # Replace with the actual path to your 'data.txt' file
string_list, number_list = f.read_data_file(data_file_path)
score_list = np.array(number_list)/1000
vector_list = np.array([f.string_to_numbers(string) for string in string_list]) # one hot encoding

In [6]:
X, Y, X_validation, Y_validation = f.create_validating_set(vector_list, score_list, percentage=0.1)

In [7]:
# Define the dataset
input_data = ptm.torch.tensor(X) 
target_data = ptm.torch.tensor(Y).view(-1, 1)

# Define the validation set
input_validation = ptm.torch.tensor(X_validation)
target_validation = ptm.torch.tensor(Y_validation).view(-1, 1)

In [8]:
input_data.shape, target_data.shape, input_validation.shape, target_validation.shape

(torch.Size([9000, 4]),
 torch.Size([9000, 1]),
 torch.Size([1000, 4]),
 torch.Size([1000, 1]))

# Training

In [9]:
for iii in range(5):

    for block_n_layers in [1,2,3,4,5,10,20,30]:
        print(iii, " # ------- block_n_layers: ", block_n_layers, " ------- #")
        print("")
        print("")

        # -- QUANTUM NODE -- #

        n_aminoacids = len(string_list[0])

        quantum_layer = qn.circuit(
            n_qubits = n_aminoacids, 
            device = "default.qubit.torch",
            device_options = {'shots': None},
            embedding = qn.parts.AngleEmbedding, 
            # embedding_ansatz = qn.parts.Ansatz_11,
            block_ansatz = qn.parts.Ansatz_11,
            final_ansatz = qn.parts.Ansatz_final_11,
            measurement = qn.parts.Measurement('Z', (n_aminoacids-1)//2),
            # embedding_n_layers = 4,
            # different_inputs_per_layer = True,
            block_n_layers = block_n_layers,
            # wrapper_qlayer = pw.QLayerEmpty,
        )

        # -- HYBRID MODEL -- #
        input_dim = input_data.size(1)

        n_pre_classical_layers = 4
        layers_dim = np.linspace(n_aminoacids, input_dim, 4).astype(int)

        import torch.nn as nn
        import torch
            
        class Reshape(nn.Module):
            def __init__(self):
                super(Reshape, self).__init__()

            def forward(self, x):
                dims = len(x.shape)
                return torch.transpose(x, dims-2, dims-1).reshape(x.shape[-3] if dims==3 else 1, x.shape[-2]*x.shape[-1]).squeeze()

        layers =  [ptm.nn.Embedding(num_embeddings=19, embedding_dim=quantum_layer.input_shape[0]//n_aminoacids), Reshape()]
        layers += [quantum_layer(), ptm.nn.Linear(1,1)]

        # Create model and set data:
        model = ptm.pytorch_model(
            layers,    
            save_options = {'initial_path': initial_path, 'name_notebook': name_notebook},
        )
        model.set_data(
            data_X=input_data, 
            data_Y=target_data, 
            data_X_validation=input_validation,
            data_Y_validation=target_validation
        )
        model.keep_track_params=False


        # -- TRAINING -- #
        import torch.optim as optim

        # train the model
        model.train(
            optimizer= optim.Adam,
            optimizer_options={'lr': 0.05},
            num_epochs = 30,                
            batch_size = 32,
            stop_training_options = {
                'lookback_epochs': 5, 
                'threshold': 0.05
            },
        )

        # save string of the model
        model.save_str(
            metadata={
                "data_file_path": data_file_path,
                "Quantum_circuit": quantum_layer.__str__(),
            }
        )

        # plot the losses of the trainig loop and save the figures
        model.plot_losses(save=True, save_txt=True)

        #save all the parameters of the model and the intermediate parameters of the model
        model.save_state_dict(intermediate=False)

        # plot the validation and save it
        model.plot_validation(save=True, fig_size=(9,9), percentatge=1)

        # print validation and save it
        model.print_validation(save=True, precision=4, percentatge=1)

0  # ------- block_n_layers:  1  ------- #


Epoch [0/100], Loss: 2.2519, Loss validation: 2.2985


/usr/lib/python3/dist-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(


	 Validation string, 	 i: 0; 	 prediction: 0.0517, 	 target: 0.0457, 	 loss: 0.1292
	 Validation string, 	 i: 1; 	 prediction: 0.0333, 	 target: -0.0040, 	 loss: 9.2233
	 Validation string, 	 i: 2; 	 prediction: -0.0359, 	 target: 0.0494, 	 loss: 1.7283
Epoch [1/100], Loss: 0.5624, Loss validation: 0.3950, Time remaining: ~0.0h 8.0m 53s
	 Validation string, 	 i: 0; 	 prediction: 0.0464, 	 target: 0.0457, 	 loss: 0.0143
	 Validation string, 	 i: 1; 	 prediction: 0.0585, 	 target: -0.0040, 	 loss: 15.4541
	 Validation string, 	 i: 2; 	 prediction: -0.0617, 	 target: 0.0494, 	 loss: 2.2499
Epoch [2/100], Loss: 0.4006, Loss validation: 0.4119, Time remaining: ~0.0h 8.0m 17s
	 Validation string, 	 i: 0; 	 prediction: 0.0924, 	 target: 0.0457, 	 loss: 1.0208
	 Validation string, 	 i: 1; 	 prediction: 0.0594, 	 target: -0.0040, 	 loss: 15.6628
	 Validation string, 	 i: 2; 	 prediction: -0.0572, 	 target: 0.0494, 	 loss: 2.1580
Epoch [3/100], Loss: 0.4058, Loss validation: 0.4137, Time remaini

KeyboardInterrupt: 

In [ ]:
# push changes to git
if initial_path != '../':
    !cd peptide-QML && git add . && git commit -m "data trained model" && git push

[main 5e76f65c] data trained model
 14 files changed, 1042 insertions(+), 1 deletion(-)
 create mode 100644 checkpoints/0809/models/0809-sh_em_1qm_v2_3.pth
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_losses_batches_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_losses_epoch_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_losses_epoch_validation_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_parameter_0.weight_None_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_parameter_2.block_weights_None_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_parameter_2.embedding_weights_None_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_parameter_3.bias_None_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_parameter_3.weight_None_3.png
 create mode 100644 checkpoints/0809/plots/0809-sh_em_1qm_v2_validation_2.png
 create mode 100644 checkpoints/0809/plots